## Deliverable 2. Create a Customer Travel Destinations Map.

In [129]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [141]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bluff,NZ,2022-07-16 19:18:17,-46.6000,168.3333,44.80,78,100,15.19
1,1,Kokstad,ZA,2022-07-16 19:20:20,-30.5472,29.4241,50.50,46,0,4.21
2,2,Luderitz,NaN,2022-07-16 19:20:20,-26.6481,15.1594,68.04,38,0,2.01
3,3,Tsibanobalka,RU,2022-07-16 19:20:21,44.9803,37.3438,71.56,83,0,2.24
4,4,Avera,US,2022-07-16 19:20:08,33.1940,-82.5271,90.55,46,38,3.60


In [142]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


In [143]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_temp_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
7,7,Dunmore Town,BS,2022-07-16 19:20:22,25.5000,-76.6500,82.83,78,99,15.26
10,10,Kapaa,US,2022-07-16 19:20:23,22.0752,-159.3190,80.58,82,100,14.97
14,14,Hilo,US,2022-07-16 19:20:24,19.7297,-155.0900,83.28,94,100,5.75
16,16,Atuona,PF,2022-07-16 19:20:25,-9.8000,-139.0333,77.38,74,74,16.04
20,20,Ponta Do Sol,PT,2022-07-16 19:20:27,32.6667,-17.1000,75.00,79,35,8.19
23,23,Sibolga,ID,2022-07-16 19:20:27,1.7427,98.7792,76.21,80,90,3.87
27,27,Puerto Asis,CO,2022-07-16 19:20:28,0.5133,-76.5007,85.98,61,93,0.38
31,31,Porto Novo,BJ,2022-07-16 19:20:30,6.4965,2.6036,75.79,86,73,9.13
32,32,Ribeira Grande,PT,2022-07-16 19:20:30,38.5167,-28.7000,75.60,78,20,11.50
35,35,Sompeta,IN,2022-07-16 19:20:31,18.9333,84.6000,80.10,89,100,8.72


In [144]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull().sum()

City_ID       0
City          0
Country       5
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [145]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
try:
    clean_df=city_data_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_df=city_data_df

In [147]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bluff,NZ,44.80,-46.6000,168.3333,
1,Kokstad,ZA,50.50,-30.5472,29.4241,
3,Tsibanobalka,RU,71.56,44.9803,37.3438,
4,Avera,US,90.55,33.1940,-82.5271,
5,Narsaq,GL,52.02,60.9167,-46.0500,
6,Toropets,RU,53.42,56.4978,31.6353,
7,Dunmore Town,BS,82.83,25.5000,-76.6500,
8,Ancud,CL,42.22,-41.8697,-73.8203,
9,Port Alfred,ZA,67.30,-33.5906,26.8910,
10,Kapaa,US,80.58,22.0752,-159.3190,


In [152]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
    results = response['results']   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index,"Hotel Name"]=np.nan    
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

In [153]:
# 7. Drop the rows where there is no Hotel Name.
try:
    clean_hotel_df=hotel_df.dropna()
    
except:
    print('No empty rows to drop')
    clean_hotel_df=hotel_df

In [154]:
# Print Clean hotel table.
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bluff,NZ,44.80,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
1,Kokstad,ZA,50.50,-30.5472,29.4241,Majestic Bed and Breakfast
3,Tsibanobalka,RU,71.56,44.9803,37.3438,Guest House Briz
5,Narsaq,GL,52.02,60.9167,-46.0500,Sleep In Narsaq
6,Toropets,RU,53.42,56.4978,31.6353,Toropa
7,Dunmore Town,BS,82.83,25.5000,-76.6500,Coral Sands Hotel
8,Ancud,CL,42.22,-41.8697,-73.8203,Hotel Arena Gruesa
9,Port Alfred,ZA,67.30,-33.5906,26.8910,The Halyards Hotel
10,Kapaa,US,80.58,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Maryborough,AU,57.45,-25.5333,152.7000,Blue Shades Motel


In [174]:
# 8a. Create the output File (CSV)
output_data_file='../Resources/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [156]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt><b>Weather</b></dt><dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [158]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [166]:
# Save fig
import matplotlib.pyplot as plt
plt.savefig("../WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>